# **온라인 강의 플랫폼 구축하기**
**과제3번** <br>

1708164 김종호


# 과제3


---
요구사항: Table 1은 코사인 유사도(Cosine similarity) 코드이다. 우리는 객체지향 디자인을 준수하여
해당 Code를 이미 개발된 LR 클래스에 적절하게 수정하여 추가해야한다. 또한, 우리는 함수
get_cosine_similarity(vec1, vec2)의 Worst-Case Time Complexity 𝑂(𝑛)를 결정해야 한다.
- 협업 필터링(Collaborative Filtering)은 아이템간 유사도 측정지표에 기반하여 아이템을 추천
한다. 협업 필터링의 예제로, 코사인 유사도1
 는 두 벡터의 내적 사이의 유사도를 측정한다. 따
라서, 두 벡터의 방향이 완전히 동일한 경우에 코사인 유사도는 1이며, 반대 방향인 경우는 -1
을 가진다. Table 1은 두 문자열로부터 코사인 유사도를 계산하는 코드이다.

- 객체지향 디자인을 준수하여 제시된 코드를 LR 클래스에 적절하게 반영하여라.
- 주어진 함수 get_cosine_similarity(vec1, vec2)의 Worst-Case Time Complexity
를 결정하라.
- 다음 요구하는 함수에 대한 코드를 작성하시오. 주어진 강좌와 유사한 강의 TOP k개를 추천
하는 함수 recommend_lectures(lec, k)를 LR 클래스 내에서 작성하시오. 인자
(Argument) lec는 주어진 강좌이며 Lecture 클래스이고, k는 주어진 강좌와 내림차순으로 높
은 코사인 유사도를 가진 강좌 k개를 가리킨다. 여기서, 유사한 강좌는 강좌의 짧은 소개가 서
로 유사한 경우를 의미하며, 서로 동일한 강좌 아이디간 유사도는 배제한다. 예로, TOP10은 강
좌의 짧은 소개(short description)간 코사인 유사도 비교 측면에서 주어진 강좌에 대해 가장
큰 코사인 유사도 값을 가진 10개의 강좌를 반환한다.

In [7]:
from typing_extensions import ParamSpecArgs
import json
import math
import re
from collections import Counter

# 정규표현식을 위한 WORD
WORD = re.compile(r"\w+")

#전체출력
def print_all(head,data_container):
 if(data_container=='SLL'): #Singly Linked List
    curNode=head
    count=0
    while curNode is not None: 

      print(curNode.name)
      curNode=curNode.next
      count=count+1
 elif(data_container=='CLL'): #Circular Linked List
  #  count=0
   curNode=head
   done = curNode is None
   while not done:
    #  count=count+1
     print(curNode.name)
     curNode=curNode.next 
     done=curNode is head
  #  print("count?",count)


def text_to_vector(text):
  '''
  텍스트를 단어(word)로 나눈뒤, 각 단어별로 카운터하여 Dict 형태로 저장
  @text : 텍스트
  '''
  words = WORD.findall(text)
  return Counter(words)

class OCP:
  '''
   Online Course Platform 클래스
  '''
  # ocp는 IMDF, LDM, LR, PM 의 클래스의 인스턴스 포함.

  def __init__(self):
    self.LR=LR()
    self.IMDF=IMDF()
    self.LDM=LDM()
    self.PM=PM()
    pass

  def loads(self,filename,data_container): # 추가
    LDM.store_lectures_from_json(self,filename,data_container,self.IMDF)
    
  def search(self,target): #검색
    LDM.search(self,target,self.IMDF)


  def delete(self, target): #삭제
    LDM.delete(self,target,self.IMDF)

class Lecture:
  '''
   OCP 플랫폼 내부의 정보교환을 위한 기본 데이터 모델
  '''
  #생성자
  def __init__(self, blocks_url,id,name,data):
    self.blocks_url=blocks_url # 강좌 URL
    self.id=id #강좌 아이디
    self.name=name #강좌명
    self.data=data # 각 강좌별 데이터
    self.next=None # 다음 노드에 대한 link

  #강좌 아이디, 강좌명과 강좌 URL 출력
  def __str__(self):
    return f" 강좌 URL: { self.blocks_url}, 강좌아이디 : {self.id}, 강좌명 : {self.name} "


class IMDF:
  '''
  인메모리 데이터 패프릭
  강의 데이터를 관리하는 다양한 종류의 데이터 구조를 제공하는 컴포넌트
  '''
  # head_reference=None
  def __init__(self):
   self.head_reference=None
   pass


class LDM:
  head=None
  '''
  대응되는 데이터 구조에 따라 강의데이터를 검색,정렬,추가,삭제가 가능한 컴포넌트
  '''
  def __init__(self):
    pass

  def store_lectures_from_json(self,filename,data_container,IMDF): #추가
    with open(filename,'r',encoding="UTF-8") as f:
     self.result= json.load(f) #200개 data
    
    #json 파일을 읽으면, 딕셔너리 형태로 변수에 저장된다.

    # print(self.IMDF.head_reference.id)
    if(data_container=='SLL'):
      IMDF.head_reference=Lecture(self.result[199]['blocks_url'],self.result[199]['id'],self.result[199]['name'],self.result[199])
      for i in range(198,-1,-1):
        newNode=Lecture(self.result[i]['blocks_url'],self.result[i]['id'],self.result[i]['name'],self.result[i])
        newNode.next=IMDF.head_reference
        IMDF.head_reference=newNode  
     #print(self.result[0]['id'])
     
    elif(data_container=='CLL'): #circular linked list
        IMDF.head_reference=Lecture(self.result[0]['blocks_url'],self.result[0]['id'],self.result[0]['name'],self.result[0])
        copy_head_reference=IMDF.head_reference
        IMDF.head_reference.next=IMDF.head_reference   
        for i in range(1,200):
          newNode=Lecture(self.result[i]['blocks_url'],self.result[i]['id'],self.result[i]['name'],self.result[i])
          # print(newNode.name)
          IMDF.head_reference.next=newNode
          newNode.next=copy_head_reference
          IMDF.head_reference=newNode
        # print(IMDF.head_reference.next.name)
          
  
 
  def search(self,name,IMDF): #검색
    curNode=IMDF.head_reference

    while curNode is not None and curNode.name!=name:
      curNode=curNode.next
    
    if curNode is None:
      print(name+ "이란 강좌명은 없습니다.")
      return
    else:
      print("강좌명: "+curNode.name +", 강좌URL: "+ curNode.blocks_url+ ", 강좌ID: "+curNode.id)
    


  def delete(self,name,IMDF):
    preNode=None
    curNode=IMDF.head_reference 

    while curNode is not None and curNode.name!=name:
      preNode=curNode
      curNode=curNode.next

    if curNode is not None:
      if curNode is IMDF.head_reference:
        IMDF.head_reference=curNode.next
      else:
        preNode.next=curNode.next
  
        
# sl=LDM()
# sl.store_lectures_from_json('/content/sample_data/lectures.json')

class LR:
 '''
 강의 데이터 조작 모듈과 함께 적절한 강의 데이터를 추천하는 컴포넌트
 '''
 def get_cosine_simiarity(self,lecture1, lecture2): 
    '''
    코사인 값을 가져오는 함수
    @vec1: 벡터 1
    @vec2: 벡터 2
    '''
    lecture1=text_to_vector(lecture1)
    lecture2=text_to_vector(lecture2)

    intersection = set(lecture1.keys()) & set(lecture2.keys())
    numerator = sum([lecture1[x] * lecture2[x] for x in intersection])
    sum1 = sum([lecture1[x] ** 2 for x in list(lecture1.keys())])
    sum2 = sum([lecture2[x] ** 2 for x in list(lecture2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
      return 0.0
    else:
      return float(numerator) / denominator 
  
 def __init__(self):
   pass

 def recommend_lectures(self,lec, k):  #주어진 강좌와 유사한 강의 TOP k개를 추천하는 함수 
   lecture_dict={}
   curNode=lec
   done = curNode is None
   while not done:
      curNode=curNode.next
      lecture2=curNode.data['short_description']
      consine=self.get_cosine_simiarity(lec.data['short_description'],lecture2)
      if consine==1.0: # 동일 강좌 배제
        done=curNode is lec
        continue
      else:
        lecture_dict[consine]=curNode.name
        done=curNode is lec

   sdict = sorted(lecture_dict.items(),reverse=True)
   count=0
   answer=[]
   for key ,value in sdict:
    #  print(key) 유사도확인
     if count==k:
       break
     answer.append(value)
     count=count+1
   return answer
  #  print(lecture_dict.values())
  #  for i in lecture_dict:
  #    print(i)
    

class PM:
  '''
  OCP 플랫폼에서 동작하는 컴포넌트의 성능을 수행시간 측면에서 모니터링하고 가장 성능이 좋은 알고리즘을 확인
  '''
  def Monitoring_component(self,program):
    '''
    @param program : 성능을 측정할 프로그램
    '''
  def __init__(self):
   pass  



In [8]:
OCP1=OCP() #OCP는 LDM과 함께 데이터 추가 및 검색 삭제의 예제로써 loads, search, delete 함수를 갖는다.
OCP1.loads('/content/sample_data/lectures.json','CLL')
cosine = OCP1.LR.get_cosine_simiarity(OCP1.IMDF.head_reference.next.data['short_description'],OCP1.IMDF.head_reference.next.next.data['short_description'])
print("Cosine Similarity:", cosine)
OCP1.LR.recommend_lectures(OCP1.IMDF.head_reference.next.next.next,15)
# print(OCP1.IMDF.head_reference.data['short_description'])
# print(OCP1.IMDF.head_reference.next.next.data['short_description'])


Cosine Similarity: 0.02823912473624525


['우리말 형태론',
 '정보통신 보안',
 '태국어 문자 배우기',
 '차이나 허스토리',
 '차이나 허스토리',
 '발달장애와 직업진로상담',
 '터키어 읽기 A2',
 '인도네시아어 A1',
 '일반화학 개념의 핵심',
 '생활원예',
 '디지털 프라이버시와 거버넌스',
 '크메르어 문자와 기초회화',
 '한국근대사, 감춰진 이야기',
 '한국근대사, 감춰진 이야기',
 '생활속의 응급처치']